# Convolutional Neural Network

### Importing the libraries 


In [1]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing (image augmentation)

### Preprocessing the training set
- rescale: _feature scalling_ because that's the representation range, this is a normalization 
- shear_range: is
- zoom_range = applies zoom over the images
- horizontal_flip = 


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, 
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
    'C:/Users/Asus/Desktop/programitas/CatDogClassifier/dataset/dataset/training_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'C:/Users/Asus/Desktop/programitas/CatDogClassifier/dataset/dataset/test_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN


### Initialising the CNN


In [10]:
cnn = tf.keras.models.Sequential()

### Setp 1 - Convolution

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size= 3, activation = 'relu',input_shape=[64,64,3]))

### Step 2 - Pooling

In [13]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))

### Adding a second convolutional layer


In [14]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size= 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))

### Step 3 - Flattening

In [15]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection


In [17]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer 

In [19]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN


### Compiling the CNN


In [20]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the training set an evaluation it on the test set


In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25,)

Epoch 1/25
250/250 [==============================] - 93s 367ms/step - loss: 0.6648 - accuracy: 0.5911 - val_loss: 0.6395 - val_accuracy: 0.6425
Epoch 2/25
250/250 [==============================] - 44s 174ms/step - loss: 0.6069 - accuracy: 0.6693 - val_loss: 0.5980 - val_accuracy: 0.6840
Epoch 3/25
250/250 [==============================] - 44s 174ms/step - loss: 0.5653 - accuracy: 0.7056 - val_loss: 0.5435 - val_accuracy: 0.7355
Epoch 4/25
250/250 [==============================] - 44s 177ms/step - loss: 0.5208 - accuracy: 0.7368 - val_loss: 0.5128 - val_accuracy: 0.7435
Epoch 5/25
250/250 [==============================] - 45s 182ms/step - loss: 0.5031 - accuracy: 0.7511 - val_loss: 0.4856 - val_accuracy: 0.7760
Epoch 6/25
250/250 [==============================] - 45s 180ms/step - loss: 0.4897 - accuracy: 0.7563 - val_loss: 0.5158 - val_accuracy: 0.7440
Epoch 7/25
250/250 [==============================] - 40s 159ms/step - loss: 0.4664 - accuracy: 0.7760 - val_loss: 0.5131 - val_ac

## Part 4 - Making a single prediction


In [32]:
import numpy as np
from keras import utils

test_image = utils.load_img('C:/Users/Asus/Desktop/programitas/CatDogClassifier/dataset/dataset/test_set/cats/cat.4034.jpg',target_size = (64,64))
test_image = utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 41ms/step


In [33]:
print(prediction)

cat
